In [1]:
import nengo
import numpy as np
import scipy.stats as st
from neuron_models import IA, InputGatedMemory
#################################
# assuming that maximum possible age is 120 yrs
max_age = 120

# our function domain is thetas (i.e., age from 1 to 120)
# Remember thetas is same as Z in our plate notation (shown later)
# We are also assuming discrete values for age, because people 
# usually report integers for age when asked to predict.
thetas = np.linspace(start=1, stop=max_age, num=max_age)

# compute likelihood 
def likelihood(x):
    x = int(x)
    like = np.asarray([1/p for p in thetas])
    like[0:x-1] = [0]*np.asarray(x-1)
    return like

# computer prior
def skew_gauss(skew, loc, scale):
    prior = [(st.skewnorm.pdf(p, a=skew, loc=loc, scale=scale)) for p in thetas] 
    prior = prior/sum(prior)
    return prior

# computer posterior
def posterior(x, skew, loc, scale):
    post = likelihood(x=x)*skew_gauss(skew=skew, loc=loc, scale=scale)
    post = post/sum(post)
    return post

#################################
# These two functions are used to generate the sample X. 

# Gaussian lifespans
skew_original = -6    # fixed parameter
loc_original = 99    # to be estimated
scale_original = 27   # fixed parameter


# Function to sample discrete random values from a skewed gaussian distribution
def randn_skew(n_samples, skew=0.0, loc=0.0, scale=1.0):
    # gaussian lifespans
    probs =  skew_gauss(skew, loc, scale) 
    samples = np.random.choice(thetas, size=n_samples, replace=True, p=probs)   
    samples = list(samples)  #convert ndarray to a python list
    return samples


# Function to draw samples X for the given number of trials
# prior gives the total lifespan, but x (samples) are current ages observed
# so x should be <= total lifespan
def draw(n_trials, n_samples):
    x_vector = []
    for i in np.arange(n_trials):
        # generating Z from alpha
        z_vector = randn_skew(n_samples=n_samples, skew=skew_original, loc=loc_original, scale=scale_original)  
        x_vector.append(np.asarray([np.random.randint(low=1, high=th+1) for th in z_vector]))   # X from Z
    return x_vector
#################################
### Construct posterior
lifespan_pred_index = 24  #6
pred_index = lifespan_pred_index

N = 108/3  #27/3
M = N 


# search space 54
alpha = np.random.randint(89, 109, M)    #loc or shift
alpha2 = np.random.randint(25, 35, M)     #scale 
alpha3 = np.random.randint(-8, -4, M)    #skew  )

prior = np.zeros((M, max_age))
    

j = 0       
for _ in range(M):
    prior[j, :] =  skew_gauss(skew=alpha3[j], loc=alpha[j], scale=alpha2[j]) #new_prior[j,:] #
    j = j+1

    
#manually add the optimal prior
prior[lifespan_pred_index,:] = skew_gauss(-6, 99, 27) 

# Search Space
# priors conditioned on alpha: p(Z/A)
pZA = prior
logpZA = np.log(pZA)  
print "Prior Space size:" , M

#################################
#Define function space for representing priors.

#Priors have values of the order of e-6 so we need to provide special encoders and eval points.
#Furthermore, priors are very high dimensional, so it will be nice to compress them to a lower dimension as well

def prior_space():
    idx = np.random.randint(0, len(pZA))
    return pZA[idx]

import function_space
nengo.dists.Function = function_space.Function
nengo.FunctionSpace = function_space.FunctionSpace

n_basis = 20

space = nengo.FunctionSpace(
        nengo.dists.Function(prior_space),
        n_basis=n_basis)
        
from copy import deepcopy
space_raw = deepcopy(space.space)

#################################
import math

# Function to compute the likelihood of each  
# 'x' in the Sample X (x_vector)
def compute_lik(x_vector):
    lik = np.zeros((len(x_vector), max_age))
    i = 0
    for obs in x_vector:
        lik[i,:] = likelihood(obs)    #pXZ
        i = i+1
    return lik   
    
global p, i, k
global spy_x
k = 0
i = 0
beta = 0.65
dt = 0.001
srch_space = M

num_observations = 5#0
# initial input
X = draw(1, num_observations+1)
_pXZ = compute_lik(X[0])
p = _pXZ[0]

a =  np.random.randint(0, M) 
L = np.zeros(M)   
pxz = p * pZA[a, :]   # pZA changing every one second, p (likelihood) changes with each incoming sample 
input = (pxz / np.sum(pxz))

model = nengo.Network(label='learn prior') 
with model:
    
    def ctx_drive(t, x):
        "t - current time in ms"
        "x - cortical state in cortex2 ensemble"
        global k, p, i, spy_x
        change_freq = 3
        
        # reconstruct the prior back to 120dim space
        # normalize reconstructed prior
        # x = np.dot(x, space._basis.T)   # gives the same result as below
        x = space.reconstruct(x)
        if sum(x) != 0:
            x = x/np.sum(x) 
        
        # swap
        if t%1<0.5:
            if t<1:
                spy_x = x
            else:    
                temp = x
                x = spy_x
                spy_x = temp
        
        # every 'change_freq' iterations, a new sample comes in (so the likelihood changes)
        if t%change_freq == 0 and t!=0:
            i = i+1
            p = _pXZ[i]
        
        # updating every 1 second, so each iteration is 1s 
        if t%1 == 0 and t!=0:
            k = k+1         
        
        nk = np.power(float(np.floor(k)+2), -beta)
        if t<1:
            return nk*input
             
        # every 1 second, a new prior (from previous iteration) is used to update ctx
        # p is the likelihood and x is the prior selected from previous iteration
        pxz = p * x   
        if pxz.any():
            pxz = pxz / np.sum(pxz)
        else:
            # pxz has all zeros
            #print "pxz = zero"
            pass
         
        return nk*pxz
     
            
    def ctx_to_bg(t, x):
        # scale x such that all  
        # why does x sometimes have all equal elements??
        if x.any():
            new_xmax = 1
            new_xmin = 0
            x_min = np.min(x)
            x_max = np.max(x)
            if x_min != x_max:
                x = (new_xmax - new_xmin)/(x_max - x_min)*(x - x_min)+ new_xmin 
            else:
                #all elements in the list are equal
                x = x/len(x)
        return x
    
    
    ### ------------------------------------------------------------------------------------------------------- ###  
    # Cortical Processing
    
    
    # node providing cortical input to the model - sample observations as sensory input 
    cortex_in = nengo.Node(output=ctx_drive, size_out=max_age, size_in=space.n_basis)
    
    # Cortex representing the summary statistics of current and previous iterations
    # This is the input for WTA network and the Memory network
    # This is also where addition of current and previous statistics takes place
    ensemble_ctx = nengo.networks.EnsembleArray(n_neurons=50, n_ensembles=srch_space, neuron_type=nengo.LIF()) 
    for ens in ensemble_ctx.all_ensembles:
        ens.encoders = nengo.dists.Choice([[-1.]]) 
        ens.eval_points = nengo.dists.Uniform(-1, 0.0)
        ens.radius = 1
        ens.intercepts=nengo.dists.Uniform(-0.6, 1.)
        
        
    # connect sensory input to cortex
    # logpZA.T => (srch_space, max_age) where max_age = 120
    # Divide by 10 to bring it in the representational range of neurons
    # 10 is just a constant scaling factor and doesn't impact the algorithm
    nengo.Connection(cortex_in, ensemble_ctx.input, transform=logpZA/10.0)
     
    
    ### ------------------------------------------------------------------------------------------------------- ###  
    # Winner take all network - Independant Accumulator
    # needs to be reset through inhibition before every iteration.
    
    wta = IA(d=srch_space, n_neurons=150, dt=dt)
    wta.x.add_neuron_input()

    def inhib(t):
        if t%1 > 0.5:
            return 2.0 
        else:
            return 0
    
    #Connecting inhibit population to error population
    inhibit = nengo.Node(inhib)
    nengo.Connection(inhibit, wta.x.neuron_input, 
                     transform=[[-2]] * wta.x.n_neurons_per_ensemble * wta.x.n_ensembles, synapse=0.01)
    
    
    ### ------------------------------------------------------------------------------------------------------- ###  
    # Prepare input for wta network
    # Input should be positive and scaled to span the entire range from 0-1 
    # in order to space apart the competing values
    
    
    # node providing a constant bias of 1 to convert the values in 'ensemble_ctx' 
    # from -ve to +ve before wta input
    node_const = nengo.Node(output=np.ones(srch_space), size_out=srch_space)
    
    # node used for scaling the values in 'ensemble_ctx' to span entire range from 0-1
    scale_node = nengo.Node(output=ctx_to_bg, size_in=srch_space, size_out=srch_space)
    
    # node to add the bias before scaling
    addbias_node = nengo.Node(size_in=srch_space, size_out=srch_space)
    nengo.Connection(node_const, addbias_node)
    nengo.Connection(ensemble_ctx.output, addbias_node)
        
    # scale the values only after the bias has been added
    nengo.Connection(addbias_node, scale_node) 
    nengo.Connection(scale_node, wta.input)

    
    ### ------------------------------------------------------------------------------------------------------- ###  
    

    # This leads to (1-n(k))u
    def mem_to_ctx(t, x):
        if t<1:
            return 0
        global k
        nk = np.power(float(np.floor(k)+2), -beta)
        return x * (1 - nk) 
      
   
    # gating signal for memory unit 2
    def gate_mem2(t, x):
        "gate 0 - gate is open"
        "gate 1 - gate is closed"
        "close the gate in first half, open in second"
        gate_input = 1
        if t%1 >= 0.5 and t%1 < 1:
            gate_input = 0
        elif t%1 >= 0 and t%1 < 0.5:
            gate_input = 1
        return gate_input 
    
    
    # gating signal for memory unit 1
    def gate_mem1(t, x):
        "gate 0 - gate is open"
        "gate 1 - gate is closed"
        "open the gate in first half, close in second"
        gate_input = 1
        if t%1 >= 0.5 and t%1 < 1:
            gate_input = 1
        elif t%1 >= 0 and t%1 < 0.5:
            gate_input = 0
        return gate_input
    
    
    ### ------------------------------------------------------------------------------------------------------- ###  
    # Memory Network
    
    
    # build two difference integrator units
    memory1 = InputGatedMemory(n_neurons=100, n_neurons_diff=30, dimensions=srch_space) 
    memory2 = InputGatedMemory(n_neurons=100, n_neurons_diff=30, dimensions=srch_space)
    
    # provide gating signal to unit 1
    gate_in1 = nengo.Node(output=gate_mem1, size_out=1, size_in=1)
    nengo.Connection(gate_in1, memory1.gate)
    
    # provide gating signal to unit 2
    gate_in2 = nengo.Node(output=gate_mem2, size_out=1, size_in=1)
    nengo.Connection(gate_in2, memory2.gate)
    
    nengo.Connection(ensemble_ctx.output, memory1.input, synapse=0.02)
    nengo.Connection(memory1.output, memory2.input, synapse=0.02)
    
    # node is needed to multiply by updated (1-nk) over time
    node_mem = nengo.Node(output=mem_to_ctx, size_in=srch_space, size_out=srch_space)
    nengo.Connection(memory2.output, node_mem, synapse=0.02)  
    nengo.Connection(node_mem, ensemble_ctx.input)


    ### ------------------------------------------------------------------------------------------------------- ###  
    # Cortex updates at the end of an iteration
    # The winning prior needs to be stored n the cortex but the values
    # are too small, so we need special eval points and encoders.
    # Moreover dimensionality reduction is important here for computational efficiency
    
    
    # ensemble to store the winning prior for the current iteration
    # works perfectly in direct mode so no error due to compression
    cortex1 = nengo.Ensemble(n_neurons=200, dimensions=space.n_basis,
                         encoders=space.project(space_raw),
                         eval_points=space.project(space_raw),
                         neuron_type = nengo.LIF()
                         )  
    
    dummy_ctx = nengo.Ensemble(n_neurons=1, dimensions=max_age, neuron_type = nengo.Direct())  
    
    def project(x):
        return space.project(x)
    
    
    # pZA.T => (max_age, srch_space)
    # function is applied before the transform when both are on a connection
    # but we want it the other way around, therefor dummy_ctx (a direct mode ensemble) is used
    nengo.Connection(wta.output, dummy_ctx, transform=pZA.T, synapse=0.02)
    nengo.Connection(dummy_ctx, cortex1, function=project)
    
    # prior is sent to the node 'cortex_in', where it is processed with the likelihood  
    # based on the incomding sensory observation to determine the posterior for next iteration.
    nengo.Connection(cortex1, cortex_in)
    
    ### ------------------------------------------------------------------------------------------------------- ###  
    # Probes
    
    # wta 
    wta_doutp = nengo.Probe(wta.output, synapse=0.02)
    wta_statep = nengo.Probe(wta.x.output, synapse=0.02)
    wta_inp = nengo.Probe(wta.input, synapse=0.02)
    
    # ctx 
    cortex1_p = nengo.Probe(cortex1, synapse=0.02)
    ensctx_p = nengo.Probe(ensemble_ctx.output, synapse=0.02)
    ctx_inp = nengo.Probe(cortex_in, synapse=0.02)
    dummy_ctxp = nengo.Probe(dummy_ctx, synapse=0.02)
    
    # memory
    memory1_ip = nengo.Probe(memory1.input, synapse=0.02)
    memory1_op = nengo.Probe(memory1.output, synapse=0.02)
    memory1_gp = nengo.Probe(memory1.gate, synapse=0.02)
    memory2_ip = nengo.Probe(memory2.input, synapse=0.02)
    memory2_op = nengo.Probe(memory2.output, synapse=0.02)
    memory2_gp = nengo.Probe(memory2.gate, synapse=0.02)
    node_memp = nengo.Probe(node_mem, synapse=0.02)
   

with nengo.Simulator(model, dt=dt) as sim:       # Create the simulator
    sim.run(3*num_observations)                  # Run it for specified number of observations

Prior Space size: 36


/home/turtlebot/.virtualenvs/turtlebot/lib/python2.7/site-packages/nengo/node.py:42: UserWarning: 'Node.size_out' is being overwritten with 'Node.size_in' since 'Node.output=None'
  warnings.warn("'Node.size_out' is being overwritten with "


HtmlProgressBar cannot be displayed. Please use the TerminalProgressBar. It can be enabled with `nengo.rc.set('progress', 'progress_bar', 'nengo.utils.progress.TerminalProgressBar')`.

<IPython.core.display.Javascript object>

HtmlProgressBar cannot be displayed. Please use the TerminalProgressBar. It can be enabled with `nengo.rc.set('progress', 'progress_bar', 'nengo.utils.progress.TerminalProgressBar')`.

<IPython.core.display.Javascript object>

In [2]:
from nengo_gui.ipython import IPythonViz
IPythonViz(model)